In [ ]:
# Import local modules
import photoshop.api as ps
from photoshop import Session
import datetime
import random
import os

# Get photoshop instance.
app = ps.Application()

In [ ]:
template_psd = "keep_template_new.psd"
is_import_base_image = True
base_image = "修改为你的图片路径"

base_path = os.path.join(os.path.dirname(os.getcwd()), 'ps-automation')
template_psd_path = os.path.join(base_path, template_psd)
base_image_path = os.path.join(base_path, base_image)
output_path = os.path.join(base_path, 'outputs', base_image.split('.')[0] + '_output.png')
FLAG_INTERVAL = 0.2469444444444413

print("template_psd_path: ", template_psd_path)
print("base_image_path: ", base_image_path)
print("output_path: ", output_path)

In [ ]:
# config img data
weather = random.choice(['晴', '阴', '雨', '雪', '多云', '雷阵雨', '小雨', '小雪', '中雨', '中雪', '大雨', '大雪'])
temperature = random.randint(10, 20)

img_data = {
    'time': datetime.datetime.now().strftime("%H:%M"),
    'user_id': '你的名字',
    #stamp generate e.g. 2024/03/19 00:23 - 01:04 · 阴 · 13℃
    'stamp': f'{datetime.datetime.now().strftime("%Y/%m/%d")} {datetime.datetime.now().strftime("%H:%M")} - {datetime.datetime.now().strftime("%H:%M")} · {weather} · {temperature}℃',
    'mile': random.randint(100, 200) / 100,
    # time range from 10min to 50min formatted output e.g. 00:40:24
    'total_time': datetime.timedelta(seconds=random.randint(600, 3000)).__str__(),
    'sport_time': datetime.timedelta(seconds=random.randint(600, 3000)).__str__(),
    # speed_avg formatted as e.g. 12'34''
    'speed_avg': f'{random.randint(10, 20)}\'{random.randint(10, 59)}\'\'',
    'cal': f'{random.randint(100, 200)}',
    'climb': f'{random.randint(100, 200)}',
    'step_frequency': f'{random.randint(60,80)}',
}
flag_attr = ['cal', 'climb', 'step_frequency']
img_data

In [ ]:

# modify control switch
config = {
    'time': False,
    'user_id': False,
    'stamp': False,
    'mile': False,
    'total_time': False,
    'sport_time': False,
    'speed_avg': False,
    'cal': False,
    'climb': False,
    'step_frequency': True,
}


In [ ]:
def print_layers(layer, indent=0):
    for item in layer:
        print('  ' * indent, item.name)
        try:
            item.layers
            print_layers(item.layers, indent + 1)
        except:
            continue

with Session(template_psd_path, action="open") as psd:
    doc = psd.active_document

    if is_import_base_image:
        try:
            doc.artLayers.getByName('base_image').remove()
        except:
            print('base_image not exist')
        # base_layer = doc.artLayers.add()
        # base_layer.name = 'base_image'
        # base_layer.kind = base_layer.kind.NORMAL
        desc = psd.ActionDescriptor
        desc.putPath(psd.app.charIDToTypeID("null"), base_image_path)
        event_id = psd.app.charIDToTypeID("Plc ")  # `Plc` need one space in here.
        psd.app.executeAction(psd.app.charIDToTypeID("Plc "), desc)
        
        new_layer = doc.artLayers[0]
        new_layer.name = 'base_image'
        new_layer.translate(-new_layer.bounds[0], -new_layer.bounds[1])
        new_layer.move(doc.layers[-1], psd.ElementPlacement.PlaceAfter)
    
    layers = doc.layerSets
    for config_key in config:
        if config[config_key]:
            print(f'modify {config_key}: {img_data[config_key]}')
            layers[config_key].visible = True
            layers[config_key].artLayers['data']
            print(layers[config_key].artLayers['data'].textItem.contents,img_data[config_key])
            layers[config_key].artLayers['data'].textItem.contents = img_data[config_key]
            if config_key in flag_attr:
                deltax = layers[config_key].artLayers['data'].bounds[2] + FLAG_INTERVAL - layers[config_key].artLayers['flag'].bounds[0]
                layers[config_key].artLayers['flag'].translate(deltax, 0)
        else:
            print(f'skip {config_key}')
            layers[config_key].visible = False
    
    doc.saveAs(output_path, ps.PNGSaveOptions(), True)


In [ ]:
# purify image

from PIL import Image

img = Image.open(output_path)
img.save(output_path, quality=95, optimize=True, progressive=True)
        